In [1]:
#Trabajaremos sobre HDFS que simula ser un cluster.
#En un datalake no hay que crear esta variable con el spark context que interpreta las tareas que le defina

import pyspark
sc = pyspark.SparkContext('local[*]')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2024-06-10T19:19:55,107 WARN [Thread-4] org.apache.hadoop.util.NativeCodeLoader - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
#creamos unos datos artificales con los que vamos a trabajar en el cluster

import random
import uuid

clientes = [str(uuid.uuid4()), str(uuid.uuid4()), str(uuid.uuid4()), str(uuid.uuid4()), str(uuid.uuid4())]

def generate_transaction():
    """
    Regresa una transacción falsa, la primera columna es el número de tarjeta ofuscado, las demás
    columnas son el comercio, la acción realizada en el comercio y el monto de la acción.
    Devuelve una cadena separada por pipes (|)
    """
    comercio = ['ARENA COLISEO', 'SUPERCITO', 'RESTAURANTE EL TRABAJO']
    accion = ['RETIRO', 'COMPRA']
    
    return "%s|%s|%s|%s" % (random.choice(clientes), random.choice(comercio), random.choice(accion), random.randint(10, 10000))

In [3]:
# creamos un archivo de transacciones 

def generate_transactions(number=10000):
    """
    Regresa una lista de transacciones falsa.
    """
    txs = []
    for i in range(number):
        txs.append(generate_transaction())
    return txs

In [4]:
# creamos un RDD en memoría con método parallelize
# Genera un millón de transacciones falsas.Distribuye estas transacciones en un RDD usando el SparkContext (sc).
# Asigna este RDD a la variable txs

txs = sc.parallelize(generate_transactions(number=1000000)) #ES UNA TRANSFORMACIÓN POR ESO NO DEMORA
# acá todavía no lo generó, solo le dijo que hacer

In [5]:
#vermos los 5 primeros del millon de filas generadas en RDD EN MEMORIA DE MAQUINAS DE CLUSTER
txs.take(5) #ES UNA ACCIÓN 

2024-06-10T19:25:00,889 WARN [dispatcher-event-loop-0] org.apache.spark.scheduler.TaskSetManager - Stage 0 contains a task of very large size (32501 KiB). The maximum recommended task size is 1000 KiB.


['34220fd5-02d3-48ce-89f0-b9bd3fcd3e9a|ARENA COLISEO|RETIRO|2135',
 'f4922ec0-9318-40a1-8e50-1324e57f7c69|ARENA COLISEO|COMPRA|1820',
 '7a2abb43-6cc5-46a1-9395-c5a98a5c4c1b|RESTAURANTE EL TRABAJO|RETIRO|5816',
 '7a2abb43-6cc5-46a1-9395-c5a98a5c4c1b|ARENA COLISEO|RETIRO|7209',
 'f4922ec0-9318-40a1-8e50-1324e57f7c69|SUPERCITO|RETIRO|7626']

In [6]:
#GUARDA El millon de filas RDD que están en memori a disco duro en HDFS para después usarlo en un DataFrame
#La ruta que pongo es de HDFS porque Spark interatua sobre el cluster no localmente como lo hace Pandas que lo hace sobre el lugar del jupyte -notebook
#Generamos un archivo distribuido en particiones en el datalake

txs.saveAsTextFile("output/raw/transacciones")

2024-06-10T20:15:34,582 WARN [dispatcher-event-loop-0] org.apache.spark.scheduler.TaskSetManager - Stage 1 contains a task of very large size (32501 KiB). The maximum recommended task size is 1000 KiB.
